# 3 More About Namespaces 

### 3.1 Namespace Scopes and Search Order


Review:
- A *namespace* is a mapping from valid identifier names to objects.
  Think of it as a dictionary.

- Simple assignment (`an_identifier =`) and `del` (`del an_identifier`) of a name are namespace
  operations, not operations on objects.

Terminology and Definitions:
- A *scope* is a section of Python code where a namespace is *directly*
  accessible.

- For an *indirectly* accessible namespace you access values via dot
  notation, e.g. `p.x` or `sys.version_info.major`.

- The (*direct*) namespace search order is (from http://docs.python.org/3/tutorial):

  - The innermost scope contains local names

  - The namespaces of enclosing functions, searched starting
    with the nearest enclosing scope; (or the module if outside any
    function)

  - The middle scope contains the current module's global names

  - The outermost scope is the namespace containing built-in
    names

- All namespace *changes* happen in the local scope (i.e. in the current scope in
  which the namespace-changing code executes):

  - *name* `=` i.e. assignment
  - `del` *name*
  - `import` *name*
  - `def` *name*
  - `class` *name*
  - function parameters: `def foo`(*name*)`:`
  - `for` loop: `for` *name* `in ...`
  - except clause: `Exception as` *name*`:`
  - with clause: `with open(filename) as` *name*`:`
  - docstrings: `__doc__`


  You should never reassign built-in names..., but let's do so to
explore how name scopes work.

In [ ]:
len

In [ ]:
def f1():
    print('In f1() a line 1, len = {}'.format(len))
    def len():
        len = range(3)
        print("In f1's local len(), len is {}".format(len))
        return len
    print('In f1() at line 6, len = {}'.format(len))
    result = len()
    print('Returning result: {!r}'.format(result))
    return result

In [ ]:
f1()

In [ ]:
def f2():
    def len():
        # len = range(3)
        print("In f1's local len(), len is {}".format(len))
        return len
    print('In f1(), len = {}'.format(len))
    result = len()
    print('Returning result: {!r}'.format(result))
    return result

In [ ]:
f2()

In [ ]:
len

In [ ]:
len = 99

In [ ]:
len

In [ ]:
def print_len(s):
    print('len(s) == {}'.format(len(s)))

In [ ]:
print_len('walk')

In [ ]:
len

In [ ]:
del len

In [ ]:
len

In [ ]:
print_len('walk')

In [ ]:
pass

In [ ]:
pass = 3

Keywords at https://docs.python.org/3/reference/lexical_analysis.html#keywords

    False     class     finally   is        return
    None      continue  for       lambda    try
    True      def       from      nonlocal  while
    and       del       global    not       with
    as        elif      if        or        yield
    assert    else      import    pass
    break     except    in        raise

### 3.2 Namespaces: Function Locals

Let's look at some surprising behaviour.

In [ ]:
x = 1
def test_outer_scope():
    print('In test_outer_scope x ==', x)

In [ ]:
test_outer_scope()

In [ ]:
def test_local():
    x = 2
    print('In test_local x ==', x)

In [ ]:
x

In [ ]:
test_local()

In [ ]:
x

In [ ]:
def test_unbound_local():
    print('In test_unbound_local  ==', x)
    x = 3

In [ ]:
x

In [ ]:
test_unbound_local()

In [ ]:
x

  Let's introspect the function `test_unbound_local` to help us understand this error.

In [ ]:
test_unbound_local.__code__

In [ ]:
test_unbound_local.__code__.co_argcount  # count of positional args

In [ ]:
test_unbound_local.__code__.co_name  # function name

In [ ]:
test_unbound_local.__code__.co_names  # names used in bytecode

In [ ]:
test_unbound_local.__code__.co_nlocals  # number of locals

In [ ]:
test_unbound_local.__code__.co_varnames  # names of locals

  See "Code objects" at https://docs.python.org/3/reference/datamodel.html?highlight=co_nlocals#the-standard-type-hierarchy

In [ ]:
import dis

In [ ]:
dis.dis(test_unbound_local.__code__.co_code)

  The use of `x` by LOAD_FAST happens before it's set by STORE_FAST.


> "This is because when you make an assignment to a variable in a
> scope, that variable becomes local to that scope and shadows any
> similarly named variable in the outer scope. Since the last
> statement in foo assigns a new value to x, the compiler recognizes
> it as a local variable. Consequently when the earlier print x
> attempts to print the uninitialized local variable and an error
> results." --
> https://docs.python.org/3/faq/programming.html#why-am-i-getting-an-unboundlocalerror-when-the-variable-has-a-value


  To explore this further on your own compare these two:

  `dis.dis(codeop.compile_command('def t1(): a = b; b = 7'))`  
`dis.dis(codeop.compile_command('def t2(): b = 7; a = b'))`

In [ ]:
def test_global():
    global x
    print('In test_global before, x ==', x)
    x = 4
    print('In test_global after, x ==', x)

In [ ]:
x

In [ ]:
test_global()

In [ ]:
x

In [ ]:
test_global.__code__.co_varnames

  Note LOAD_GLOBAL instead of LOAD_FAST:

In [ ]:
dis.dis(test_global.__code__.co_code)

In [ ]:
def test_nonlocal():
    x = 5
    def test6():
        nonlocal x
        print('test6 before x ==', x)
        x = 6
        print('test6 after x ==', x)
    print('test_nonlocal before x ==', x)
    test6()
    print('test_nonlocal after x ==', x)

In [ ]:
x = 1

In [ ]:
x

In [ ]:
test_nonlocal()

In [ ]:
x